In [2]:
import numpy as np
import modern_robotics as mr


In [8]:
T = np.array([[0, 1, 0,-0.5],
              [0, 0,-1, 0.1],
              [-1,0, 0, 0.1],
              [ 0,0, 0,   1]])

M = np.array([[-1, 0, 0, 0.817],
              [0,  0, 1, 0.191],
              [0,  1, 0, -0.006],
              [0,  0, 0,     1]])

blist = np.array([[   0,    0,    0,    0,     0, 0],
                  [   1,    0,    0,    0,    -1, 0],
                  [   0,    1,    1,    1,     0, 1],
                  [.191, .095, .095, .095, -.082, 0],
                  [   0,-.817,-.392,    0,     0, 0],
                  [.817,    0,    0,    0,     0, 0]])

eomg = 0.001
ev = 0.0001

thetalist = np.array([6,-2.4,4,-5,3,1])

In [11]:
def IKinBodyIterations(blist, M, T, thetalist, eomg, ev):
    
    theta_array = thetalist
    
    Vb = mr.se3ToVec(mr.MatrixLog6(np.dot(mr.TransInv(mr.FKinBody(M,blist, thetalist)),T)))
    
    err = np.linalg.norm(Vb[0:3]) > eomg or np.lingalg.norm(Vb[3:])>ev
    
    Tsd_twist = mr.se3ToVec(mr.MatrixLog6(T))
    Tsd_eomg_mag = np.linalg.norm(Tsd_twist[0:3])
    Tsd_ev_mag = np.linalg.norm(Tsd_twist[3:])
    
    i = 0
    maxiter = 20
    
    print(f'''Iteration 0 \n
    Joint Vector {thetalist} \n
    SE3 end-effector config: \n {mr.FKinBody(M, blist, thetalist)} \n
    Error twist Vb {Vb} \n
    Angular error magnitude ||omega_b||: {Tsd_eomg_mag - np.linalg.norm(Vb[0:3])} \n
    Linear error magnitude ||v_b||: {Tsd_ev_mag - np.linalg.norm(Vb[3:])}''')
    
    while err and i < maxiter:
        
        thetalist = thetalist + np.dot(np.linalg.pinv(mr.JacobianBody(blist,thetalist)), Vb)
        theta_array = np.vstack([theta_array, thetalist])
        
        i += 1
        Vb = mr.se3ToVec(mr.MatrixLog6(np.dot(mr.TransInv(mr.FKinBody(M,blist, thetalist)),T)))
        err = np.linalg.norm(Vb[0:3]) > eomg or np.linalg.norm(Vb[3:])>ev
        
        #Find the new end-effector representation
        Tsb = mr.FKinBody(M, blist, thetalist)
        
        #Find new twist to find the current eomg and ev magnitutdes
        current_twist = mr.se3ToVec(mr.MatrixLog6(Tsb))
        current_eomg = np.linalg.norm(current_twist[0:3])
        current_ev = np.linalg.norm(current_twist[3:])
        
        #Find if the threshold has been reached
        error_omg = Tsd_eomg_mag - current_eomg
        error_v = Tsd_ev_mag - current_ev
        
        #Current Vb error twist
        Vb_error_twist = Tsd_twist - current_twist
        
        print(f'''Iteration {i} \n
        Joint Vector: {thetalist} \n
        SE3 end-effector config: \n {Tsb} \n
        Error twist Vb: {Vb_error_twist} \n
        Angular error magnitude ||omega_b||: {error_omg} \n
        Linear error magnitude ||v_b||: {error_v} \n''')
        
    np.savetxt('thetalist.csv', theta_array, delimiter=',')
        

In [12]:
answer = IKinBodyIterations(blist, M, T, thetalist, eomg, ev)
answer

Iteration 0 

    Joint Vector [ 6.  -2.4  4.  -5.   3.   1. ] 

    SE3 end-effector config: 
 [[-0.26876783  0.87270274 -0.40761967 -0.33817832]
 [ 0.15762346 -0.3776362  -0.91243945  0.12738665]
 [-0.95022034 -0.30948479 -0.03606196  0.07312775]
 [ 0.          0.          0.          1.        ]] 

    Error twist Vb [-0.41091707 -0.10135377  0.3025942  -0.00439713 -0.15689686  0.059557  ] 

    Angular error magnitude ||omega_b||: 1.5741176032456146 

    Linear error magnitude ||v_b||: 0.42900611105167097
Iteration 1 

        Joint Vector: [ 6.103204   -2.18539258  4.2128313  -6.21134064  3.30075903  0.53025893] 

        SE3 end-effector config: 
 [[-0.01732346  0.99502363 -0.09812173 -0.48041178]
 [-0.13582286 -0.09956868 -0.98571711  0.11590784]
 [-0.99058167 -0.00374886  0.13687183  0.14351787]
 [ 0.          0.          0.          1.        ]] 

        Error twist Vb: [ 0.03603207  0.14296858  0.14183427  0.02537422 -0.01669063 -0.07686809] 

        Angular error magnitud